In [2]:
# give 2 portforlio, the old one and the optimized new one, 
# calculate the stepwise change of the portfolio
# and prioritize the changes by the improvement of the portfolio

cur_investment = [
        {
            "stock": "FRAS.L",
            "quantity": 250,
            
        },
        {
            "stock": "GILD",
            "quantity": 30,
        },
        {
            "stock": "LLY",
            "quantity": 6.68,
        },
        {
            "stock": "NVR",
            "quantity": 0.52,
        },
        {
            "stock": "MCK",
            "quantity": 6.56,
        },
        {
            "stock": "HOLX",
            "quantity": 25.26,
        },
        {
            "stock": "STAN.L",
            "quantity": 196.2
        },
        {
            "stock": "JKHY",
            "quantity": 18.33
        },
        {
            "stock": "601225.SS",
            "quantity": 800
        }
    ]


In [3]:
# compute the portfolio value

from util import get_currency_pair, read_and_filter_exchange_rates

tot_value = 0
for stock in cur_investment:
    # get the price of the stock which is the last line in the stock price file
    file_path = './data/prices/' + stock["stock"] + '.csv'
    with open(file_path) as f:
        lines = f.readlines()
        stock_price = float(lines[-1].split(",")[1])
        

    if '.' in stock["stock"]:
        stock_suffix = '.' + stock["stock"].split(".")[-1]
        # convert GBX to GBP
        if stock_suffix == '.L':
            stock_price *= 0.01
        exchange_name, needs_inversion, exchange_name_yahoo = get_currency_pair(stock_suffix, "USD")
        df_rate = read_and_filter_exchange_rates(exchange_name, exchange_name_yahoo)
        #print(df_rate)
        rate = df_rate[exchange_name].iloc[-1]

        if needs_inversion:
            rate = 1 / rate

        print(stock["stock"], stock_price, stock_price * rate, stock["quantity"] * stock_price * rate)
        tot_value += stock["quantity"] * stock_price * rate
        stock['value'] = stock["quantity"] * stock_price * rate
    else:
        print(stock["stock"], stock_price, stock["quantity"] * stock_price)
        tot_value += stock["quantity"] * stock_price
        stock['value'] = stock["quantity"] * stock_price


print(tot_value)


Metric: DEXUSUK need to be refreshed...


[*********************100%%**********************]  1 of 1 completed

FRAS.L 8.665000000000001 11.373779339790346 2843.4448349475865
GILD 78.93000030517578 2367.9000091552734
LLY 955.0 6379.4
NVR 9070.0 4716.400000000001
MCK 556.7000122070312 3651.9520800781247
HOLX 80.58999633789062 2035.7033074951173
STAN.L 7.78 10.212118091583251 2003.6175695686338
JKHY 174.60000610351562 3200.4181118774413
27198.835913122173


In [4]:
# compute the weight
for stock in cur_investment:
    stock['weight'] = stock['value'] / tot_value

cur_investment

[{'stock': 'FRAS.L',
  'quantity': 250,
  'value': 2843.4448349475865,
  'weight': 0.10454288720407172},
 {'stock': 'GILD',
  'quantity': 30,
  'value': 2367.9000091552734,
  'weight': 0.08705887328114921},
 {'stock': 'LLY',
  'quantity': 6.68,
  'value': 6379.4,
  'weight': 0.23454680267850125},
 {'stock': 'NVR',
  'quantity': 0.52,
  'value': 4716.400000000001,
  'weight': 0.1734044800691105},
 {'stock': 'MCK',
  'quantity': 6.56,
  'value': 3651.9520800781247,
  'weight': 0.1342686904595144},
 {'stock': 'HOLX',
  'quantity': 25.26,
  'value': 2035.7033074951173,
  'weight': 0.07484523653870735},
 {'stock': 'STAN.L',
  'quantity': 196.2,
  'value': 2003.6175695686338,
  'weight': 0.0736655633339801},
 {'stock': 'JKHY',
  'quantity': 18.33,
  'value': 3200.4181118774413,
  'weight': 0.11766746643496565}]

In [5]:
import pandas as pd
import numpy as np

data_dir = "./processed_data_128"
S = pd.read_pickle(f'{data_dir}/S.pkl')
mu = np.load(f'{data_dir}/mu.npy')

# load valid_tickers.txt
with open(f'{data_dir}/valid_tickers.txt', 'r') as f:
    valid_tickers = f.read().splitlines()


In [6]:
current_weight = np.zeros(len(valid_tickers))
for stock in cur_investment:
    if stock["stock"] in valid_tickers:
        current_weight[valid_tickers.index(stock["stock"])] = stock["weight"]

In [7]:
def portfolio_volatility(weights, covariance_log_returns):
    covariance_returns = np.exp(covariance_log_returns) - 1
    return np.sqrt(np.dot(weights.T, np.dot(covariance_returns, weights)))

def portfolio_return(weights, log_returns, allow_short=False):
    returns = np.exp(log_returns) - 1
    return np.sum(np.abs(returns)*weights) if allow_short else np.sum(returns*weights)


In [8]:
print(portfolio_volatility(current_weight, S))
print(portfolio_return(current_weight, mu))

0.07176496461670316
0.18046479228175294


In [9]:
from scipy.optimize import minimize
import json

def portfolio_volatility_log_return(weights, covariance):
    return np.sqrt(np.dot(weights.T, np.dot(covariance, weights)))

def portfolio_log_return(weights, returns, allow_short=False):
    return np.sum(np.abs(returns)*weights) if allow_short else np.sum(returns*weights)


def min_func_sharpe(weights, returns, covariance, risk_free_rate, allow_short=False):
    portfolio_ret = portfolio_log_return(weights, returns, allow_short)
    portfolio_vol = portfolio_volatility_log_return(weights, covariance)
    sharpe_ratio = (portfolio_ret - risk_free_rate) / portfolio_vol
    return -sharpe_ratio # Negate Sharpe ratio because we minimize the function


def optimize_portfolio(returns, covariance, risk_free_rate, allow_short=False):
    num_assets = len(returns)
    args = (returns, covariance, risk_free_rate)

    # Define constraints
    def constraint_sum(weights):
        return np.sum(weights) - 1
    
    constraints = [{'type': 'eq', 'fun': constraint_sum}]

    bounds = tuple((0.0, 0.20) for _ in range(num_assets))

    # Perform optimization
    def objective(weights):
        return min_func_sharpe(weights, returns, covariance, risk_free_rate, allow_short)
    
    iteration = [0]  # mutable container to store iteration count
    def callback(weights):
        iteration[0] += 1
        
        print(f"Iteration: {iteration[0]}, value: {objective(weights)}")

    # Initial guess (equal weights)
    initial_guess = num_assets * [1. / num_assets]

    # Perform optimization
    result = minimize(objective, initial_guess, 
                      method='SLSQP', bounds=bounds, constraints=constraints, callback=callback, options={'maxiter': 300})

    return result

INTEREST_RATE = 0.0497    # Current interest rate accessible for USD
ANNUAL_TRADING_DAYS = 252

def do_optimization(mu, S, final_tickers, period, allow_short):
  riskfree_log_return = np.log(1 + INTEREST_RATE) * period / ANNUAL_TRADING_DAYS
  raw_weights = optimize_portfolio(mu, S, riskfree_log_return, allow_short)
  weights = raw_weights.x
  
  tickers_to_buy = []
  print(f'Starting optimization: allow_short: {allow_short}')
  for index, ticker_name in enumerate(final_tickers):
    weight = weights[index]
    if weight > 1e-3:
      print(f'index: {index} {ticker_name}: weight {weight} exp profit: {mu[index]}, variance: {S[ticker_name][ticker_name]}')
      ticker_info = {'id': ticker_name, 'weight': weight}
      tickers_to_buy.append(ticker_info)

  print(f'expected return in {period} trading days: {portfolio_return(weights, mu)}')
  print(f'volatility of the return in {period} trading days: {portfolio_volatility(weights, S)}')
  print(f'optimization finished for allow_short={allow_short}')
  # print tickers_to_buy in JSON format

  tickers_to_buy_json = json.dumps(tickers_to_buy, indent=4)
  print(tickers_to_buy_json)
  return tickers_to_buy

In [10]:
tickers_to_buy = do_optimization(mu, S, valid_tickers, period=128, allow_short=False)

Iteration: 1, value: -1.524015519957314
Iteration: 2, value: -1.8320812358156258
Iteration: 3, value: -0.5956994327654351
Iteration: 4, value: -1.1814796779594026
Iteration: 5, value: -1.1302735363640226
Iteration: 6, value: -1.7380584126071628
Iteration: 7, value: -1.202338705437104
Iteration: 8, value: -2.1284707430644407
Iteration: 9, value: -0.821025177058987
Iteration: 10, value: -1.4081631260436491
Iteration: 11, value: -1.2943158408494106
Iteration: 12, value: -2.7697784094491453
Iteration: 13, value: -2.6143369663213147
Iteration: 14, value: -2.1662767600599073
Iteration: 15, value: -2.7390661374935137
Iteration: 16, value: -2.6870072679403894
Iteration: 17, value: -2.4580856682497663
Iteration: 18, value: -2.890460411938861
Iteration: 19, value: -2.9556439013209768
Iteration: 20, value: -2.8506153205969307
Iteration: 21, value: -3.000161603835807
Iteration: 22, value: -2.976291883233221
Iteration: 23, value: -2.96899954083924
Iteration: 24, value: -3.030527276469221
Iteration:

In [35]:
# to find the steps to optimize my current portfolio to the optimized portfolio
# we need to find the difference between the optimized portfolio and the current portfolio
# and then prioritize the changes by the improvement of the portfolio

old_portfolio = pd.DataFrame(cur_investment)
optimized_portfolio = pd.DataFrame(tickers_to_buy)

# Merge the two dataframes to compare weights
comparison = pd.merge(old_portfolio, optimized_portfolio, left_on='stock', right_on='id', how='outer')
comparison['stock'] = comparison['stock'].fillna(comparison['id'])
comparison.fillna(0, inplace=True)  # Assuming no stock means 0 weight
comparison['weight_difference'] = comparison['weight_y'] - comparison['weight_x']

#*comparison['new_quantity'] = comparison['quantity'] * comparison['weight_y'] / comparison['weight_x'] if comparison['weight_x'] != 0 else 0
print(comparison[['stock','weight_x', 'weight_y', 'weight_difference', 'quantity']])

        stock  weight_x  weight_y  weight_difference  quantity
0      FRAS.L  0.104543  0.037291          -0.067252    250.00
1        GILD  0.087059  0.163008           0.075949     30.00
2         LLY  0.234547  0.066531          -0.168016      6.68
3         NVR  0.173404  0.000000          -0.173404      0.52
4         MCK  0.134269  0.014877          -0.119391      6.56
5        HOLX  0.074845  0.000000          -0.074845     25.26
6      STAN.L  0.073666  0.012661          -0.061005    196.20
7        JKHY  0.117667  0.200000           0.082333     18.33
8        BA.L  0.000000  0.023168           0.023168      0.00
9        AMGN  0.000000  0.004777           0.004777      0.00
10       ACGL  0.000000  0.012257           0.012257      0.00
11       CTRA  0.000000  0.032321           0.032321      0.00
12       FICO  0.000000  0.011387           0.011387      0.00
13        JBL  0.000000  0.010057           0.010057      0.00
14        MRK  0.000000  0.020639           0.020639   

In [31]:
comparison

,stock,quantity,value,weight_x,id,weight_y,weight_difference
0,FRAS.L,250.00,2843.444835,0.104543,FRAS.L,0.037291,-0.067252
1,GILD,30.00,2367.900009,0.087059,GILD,0.163008,0.075949
2,LLY,6.68,6379.400000,0.234547,LLY,0.066531,-0.168016
3,NVR,0.52,4716.400000,0.173404,0,0.000000,-0.173404
4,MCK,6.56,3651.952080,0.134269,MCK,0.014877,-0.119391
5,HOLX,25.26,2035.703307,0.074845,0,0.000000,-0.074845
6,STAN.L,196.20,2003.617570,0.073666,STAN.L,0.012661,-0.061005
7,JKHY,18.33,3200.418112,0.117667,JKHY,0.200000,0.082333
8,BA.L,0.00,0.000000,0.000000,BA.L,0.023168,0.023168
9,AMGN,0.00,0.000000,0.000000,AMGN,0.004777,0.004777


In [32]:
# Initialize the weights array

current_weight = np.zeros(len(valid_tickers))
for stock in cur_investment:
    if stock["stock"] in valid_tickers:
        current_weight[valid_tickers.index(stock["stock"])] = stock["weight"]


original_sharpe = portfolio_log_return(current_weight, mu) / portfolio_volatility_log_return(current_weight, S)
print(f'Original Sharpe ratio: {original_sharpe}')
# Optimize one stock at a time and adjust other weights proportionally

new_weights = current_weight.copy()
for i in range(10):
    results = []
    print(f'Starting iteration {i} optimization...')
    for index, row in comparison.iterrows():
        new_weights_try = new_weights.copy()
        stock_name = comparison.loc[index, 'stock']
        stock_idx = valid_tickers.index(stock_name)
        # Adjust the targeted stock weight
        new_weights_try[stock_idx] = comparison.loc[index, 'weight_y']
        # Adjust the rest to ensure total sum remains 1
        total_adjusted_weights = np.sum(new_weights_try) - new_weights_try[stock_idx]
        scale_factor = (1 - new_weights_try[stock_idx]) / total_adjusted_weights
        for j in range(len(new_weights_try)):
            if j != stock_idx:
                new_weights_try[j] *= scale_factor

        new_sharpe = portfolio_log_return(new_weights_try, mu) / portfolio_volatility_log_return(new_weights_try, S)
        results.append((comparison.loc[index, 'stock'], new_sharpe, (new_sharpe - original_sharpe)/original_sharpe, new_weights_try))

    results.sort(key=lambda x: x[2], reverse=True)


    stock_name = results[0][0]
    stock_idx = valid_tickers.index(stock_name)
    # Adjust the targeted stock weight
    new_weights = results[0][3]
    new_sharpe = results[0][1]
    # remove the row from comparison
    comparison = comparison[comparison['stock'] != stock_name]

    original_sharpe = new_sharpe

    print(f'Iteration {i} optimization result:')
    print(f'Optimized ticker: {stock_name}')
    print(f'Sharpe ratio: {results[0][1]}')
    print(f'Improvement: {results[0][2]}')
    


Original Sharpe ratio: 2.316911658068263
Starting iteration 0 optimization...
Iteration 0 optimization result:
Optimized ticker: JKHY
Sharpe ratio: 2.504765772423389
Improvement: 0.08107953261875782
Starting iteration 1 optimization...
Iteration 1 optimization result:
Optimized ticker: FRAS.L
Sharpe ratio: 2.677524361334956
Improvement: 0.06897195371063425
Starting iteration 2 optimization...
Iteration 2 optimization result:
Optimized ticker: WMT
Sharpe ratio: 2.7980000910564367
Improvement: 0.0449951946138088
Starting iteration 3 optimization...
Iteration 3 optimization result:
Optimized ticker: GILD
Sharpe ratio: 2.9006768688019453
Improvement: 0.03669648835027062
Starting iteration 4 optimization...
Iteration 4 optimization result:
Optimized ticker: 600900.SS
Sharpe ratio: 2.9772513177338693
Improvement: 0.026398820825413494
Starting iteration 5 optimization...
Iteration 5 optimization result:
Optimized ticker: VRTX
Sharpe ratio: 3.044129910435139
Improvement: 0.022463200302544287
S

In [53]:
results

[('JKHY', 2.5046672637587784, 0.08108897799664981),
 ('FRAS.L', 2.483166099644885, 0.07180843527787617),
 ('GILD', 2.423743862862956, 0.046160028336810606),
 ('WMT', 2.4118456577744456, 0.04102440870171832),
 ('VRTX', 2.3708321574377273, 0.0233217606076367),
 ('PGR', 2.367085886545834, 0.021704758529788773),
 ('CTRA', 2.3587781255117957, 0.018118881469116714),
 ('600519.SS', 2.3487388303444434, 0.013785622712825156),
 ('600276.SS', 2.3484075768041413, 0.01364264382036915),
 ('600900.SS', 2.341138647321499, 0.010505157392840131),
 ('FICO', 2.3382226724943918, 0.009246535822096853),
 ('MRK', 2.336366207672152, 0.008445230320802391),
 ('601225.SS', 2.328626235066381, 0.005104427654049095),
 ('600150.SS', 2.328414047687685, 0.005012841262635975),
 ('AMGN', 2.324616489584428, 0.003373702097071988),
 ('TTWO', 2.3200271784217326, 0.0013928187332908754),
 ('STAN.L', 2.319078275611021, 0.000983244022632122),
 ('NOC', 2.317373873661209, 0.00024757338538655005),
 ('600406.SS', 2.3161288930101893,

In [37]:
optimized_portfolio

,id,weight
0,BA.L,0.023168
1,FRAS.L,0.037291
2,STAN.L,0.012661
3,AMGN,0.004777
4,ACGL,0.012257
5,CTRA,0.032321
6,LLY,0.066531
7,FICO,0.011387
8,GILD,0.163008
9,JBL,0.010057
